In [13]:
import os
import sys
import pandas as pd
from datetime import datetime, timedelta
from sqlalchemy import create_engine
from dotenv import load_dotenv
import openpyxl
import time

# === 1. Load Environment Variables from .env ===
load_dotenv()

# Read ZID for GULSHAN TRADING

ZID_HMBR = int(os.environ["ZID_GULSHAN_TRADING"])
ZID_GI = int(os.environ["ZID_GI"])
ZID_ZEPTO = int(os.environ["ZID_ZEPTO_CHEMICALS"])


# === 2. Add root (E:\) to Python path ===
# For Jupyter notebooks, use os.getcwd() instead of __file__
CURRENT_DIR = os.getcwd()
PROJECT_ROOT = os.path.dirname(CURRENT_DIR)
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)


# === 3. Import shared modules ===
from mail import send_mail, get_email_recipients
from project_config import DATABASE_URL, holiday


# === 4. Create engine using shared DATABASE_URL ===
engine = create_engine(DATABASE_URL)


# === 5. Suppress warnings ===
import warnings
warnings.filterwarnings('ignore', category=pd.errors.DtypeWarning)
pd.set_option('display.float_format', '{:.2f}'.format)

In [14]:
def get_order_details(zid, date_param, xstate:tuple = ()):
    """Fetch order details for a specific date."""
    query = """
        SELECT 
            opdor.zid,
            opdor.xdate as delivery_date,
            opdor.xsp As SP_Code,
            prmst.xname As SP_Name,
            cacus.xcus As Customer_Code,
            cacus.xshort AS customer_name,
            cacus.xstate As State,
            cacus.xmobile As Mobile_Number
        FROM opdor
        LEFT JOIN opddt ON opdor.xdornum = opddt.xdornum
        LEFT JOIN cacus ON opdor.xcus = cacus.xcus
        LEFT JOIN prmst ON opdor.xsp = prmst.xemp
        WHERE opdor.zid = %s
        AND opddt.zid = %s
        AND cacus.zid = %s
        AND prmst.zid = %s
        AND opdor.xdate = %s
        AND (opdor.xstatusdor = '2-Confirmed' OR opdor.xstatusdor = '3-Invoiced')
        AND cacus.xstate IN %s
        GROUP BY opdor.xdornum, opdor.zid, opdor.xdate, opdor.xsp, prmst.xname, cacus.xcus, cacus.xshort,  cacus.xstate, cacus.xmobile
        ORDER BY opdor.xdornum
    """
    
    # Execute query with parameters
    df = pd.read_sql(query, engine, params=[zid, zid, zid, zid, date_param, xstate])
    return df
    

In [15]:
# get 4 days ago date for district
district_date = (datetime.now() - timedelta(days=4)).strftime('%Y-%m-%d')
print (district_date)
# get 1 days ago date for city
city_date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')

# create a reusable function to pass the zid for each business


2025-09-04


In [16]:
def get_all_business_order_data(zid):
    # Get original data
    df_order_details_district = get_order_details(zid, district_date, ('District',))
    df_order_details_city = get_order_details(zid, city_date, ('Dhaka retail', 'Dhaka General'))
    
    # Combine for sampling
    df_combined = pd.concat([df_order_details_district, df_order_details_city])
    
    # Sample 5 from 'District' — if less than 5, take all
    df_sample_district = df_combined[df_combined['state'] == 'District']
    if len(df_sample_district) >= 5:
        df_sample_district = df_sample_district.sample(5)
    else:
        print(f"Warning: Only {len(df_sample_district)} 'District' rows available.")

    # Sample 5 from 'Dhaka retail' or 'Dhaka General'
    df_sample_city = df_combined[df_combined['state'].isin(['Dhaka retail', 'Dhaka General'])]
    if len(df_sample_city) >= 5:
        df_sample_city = df_sample_city.sample(5)
    else:
        print(f"Warning: Only {len(df_sample_city)} city rows available.")

    # Concatenate the two samples
    df_final_sample = pd.concat([df_sample_district, df_sample_city], ignore_index=True)
    
    return df_final_sample

In [17]:

# get all business order data
df_order_details_HMBR = get_all_business_order_data(ZID_HMBR)

df_order_details_GI = get_all_business_order_data(ZID_GI)
# take both dataframe and remove duplicate customer if any from hmbr and gi order dataframe
# df_order_details = pd.concat([df_order_details_HMBR, df_order_details_GI])
# df_order_details = df_order_details.drop_duplicates(subset=['customer_code']).sort_values(by='customer_code')

df_order_details_GI.sort_values(by='customer_code')


,zid,delivery_date,sp_code,sp_name,customer_code,customer_name,state,mobile_number
8,100000,2025-09-07,SA--000255,Sumon Miah,CUS-000368,Asif Hardware & Paint,Dhaka retail,+8801712276968
5,100000,2025-09-07,SA--000006,Muhammad Zakir Hossain,CUS-000808,Rahaman Hardware,Dhaka retail,"01819418349,01943201084"
6,100000,2025-09-07,SA--000108,Md. Anichur Rahman,CUS-001246,Al-Amin Hardware,Dhaka retail,01712271091
2,100000,2025-09-04,SA--000144,Syed Mobarok Hossain,CUS-002683,M/S Chandgaon Hardware Store,District,"01919269644,01812347618"
3,100000,2025-09-04,SA--000070,Md. Sumon Sheikh,CUS-002863,Shanda Hardware,District,+8801711888051
0,100000,2025-09-04,SA--000021,Md. Maruful Islam,CUS-003184,Riyadh Sanitary,District,01711966560
4,100000,2025-09-04,SA--000144,Syed Mobarok Hossain,CUS-004103,Giash Traders,District,01711170968
7,100000,2025-09-07,SA--000258,Md. Suruj Mia,CUS-005016,Tilottoma Electric,Dhaka retail,
9,100000,2025-09-07,SA--000092,Md. Abdullah Al Mamun Emon,CUS-007190,Jahid Enterprise,Dhaka retail,01790988886
1,100000,2025-09-04,SA--000114,Md. Pavel Mia,CUS-007414,Tarek Hardware,District,01917681570


In [18]:
df_order_details_HMBR

,zid,delivery_date,sp_code,sp_name,customer_code,customer_name,state,mobile_number
0,100001,2025-09-04,SA--000021,Md. Maruful Islam,CUS-002820,M/s Yusuf Enterprise,District,"01710641821, 01971151844"
1,100001,2025-09-04,SA--000192,Sojib Hossen,CUS-002648,Maruf & Brothers,District,01714763219
2,100001,2025-09-04,SA--000144,Syed Mobarok Hossain,CUS-002683,M/S Chandgaon Hardware Store,District,"01919269644,01812347618"
3,100001,2025-09-04,SA--000021,Md. Maruful Islam,CUS-004865,Dhaka Fency Hardware,District,01741160257
4,100001,2025-09-04,SA--000021,Md. Maruful Islam,CUS-000400,Wasam,District,01711156534
5,100001,2025-09-07,SA--000108,Md. Anichur Rahman,CUS-007108,Bismillah Sanitary,Dhaka retail,01736482159
6,100001,2025-09-07,SA--000255,Sumon Miah,CUS-001662,Bhai Bon Hardware,Dhaka retail,01819961429
7,100001,2025-09-07,SA--000014,Md. Abdul Kuddus,CUS-000855,Mosharof Electric & Hardware,Dhaka retail,01814842364
8,100001,2025-09-07,SA--000287,Md.Ayob Hossain,CUS-007066,Liton Hardware,Dhaka retail,01816144852
9,100001,2025-09-07,SA--000258,Md. Suruj Mia,CUS-003713,Laxmipur Hardware,Dhaka retail,01711980589


In [19]:
df_order_details_GI.sort_values(by='customer_code')
df_order_details_GI


,zid,delivery_date,sp_code,sp_name,customer_code,customer_name,state,mobile_number
0,100000,2025-09-04,SA--000021,Md. Maruful Islam,CUS-003184,Riyadh Sanitary,District,01711966560
1,100000,2025-09-04,SA--000114,Md. Pavel Mia,CUS-007414,Tarek Hardware,District,01917681570
2,100000,2025-09-04,SA--000144,Syed Mobarok Hossain,CUS-002683,M/S Chandgaon Hardware Store,District,"01919269644,01812347618"
3,100000,2025-09-04,SA--000070,Md. Sumon Sheikh,CUS-002863,Shanda Hardware,District,+8801711888051
4,100000,2025-09-04,SA--000144,Syed Mobarok Hossain,CUS-004103,Giash Traders,District,01711170968
5,100000,2025-09-07,SA--000006,Muhammad Zakir Hossain,CUS-000808,Rahaman Hardware,Dhaka retail,"01819418349,01943201084"
6,100000,2025-09-07,SA--000108,Md. Anichur Rahman,CUS-001246,Al-Amin Hardware,Dhaka retail,01712271091
7,100000,2025-09-07,SA--000258,Md. Suruj Mia,CUS-005016,Tilottoma Electric,Dhaka retail,
8,100000,2025-09-07,SA--000255,Sumon Miah,CUS-000368,Asif Hardware & Paint,Dhaka retail,+8801712276968
9,100000,2025-09-07,SA--000092,Md. Abdullah Al Mamun Emon,CUS-007190,Jahid Enterprise,Dhaka retail,01790988886


In [20]:
# concat both dataframe
df_order_details = pd.concat([df_order_details_GI, df_order_details_HMBR], axis=0)
df_order_details


,zid,delivery_date,sp_code,sp_name,customer_code,customer_name,state,mobile_number
0,100000,2025-09-04,SA--000021,Md. Maruful Islam,CUS-003184,Riyadh Sanitary,District,01711966560
1,100000,2025-09-04,SA--000114,Md. Pavel Mia,CUS-007414,Tarek Hardware,District,01917681570
2,100000,2025-09-04,SA--000144,Syed Mobarok Hossain,CUS-002683,M/S Chandgaon Hardware Store,District,"01919269644,01812347618"
3,100000,2025-09-04,SA--000070,Md. Sumon Sheikh,CUS-002863,Shanda Hardware,District,+8801711888051
4,100000,2025-09-04,SA--000144,Syed Mobarok Hossain,CUS-004103,Giash Traders,District,01711170968
5,100000,2025-09-07,SA--000006,Muhammad Zakir Hossain,CUS-000808,Rahaman Hardware,Dhaka retail,"01819418349,01943201084"
6,100000,2025-09-07,SA--000108,Md. Anichur Rahman,CUS-001246,Al-Amin Hardware,Dhaka retail,01712271091
7,100000,2025-09-07,SA--000258,Md. Suruj Mia,CUS-005016,Tilottoma Electric,Dhaka retail,
8,100000,2025-09-07,SA--000255,Sumon Miah,CUS-000368,Asif Hardware & Paint,Dhaka retail,+8801712276968
9,100000,2025-09-07,SA--000092,Md. Abdullah Al Mamun Emon,CUS-007190,Jahid Enterprise,Dhaka retail,01790988886


In [21]:
# now remove duplicates if any customer_code
df_order_details = df_order_details.drop_duplicates(subset=['customer_code']).sort_values(by=['zid', 'customer_code']).reset_index(drop=True)

# add today_date column before zid column



In [22]:
df_order_details

,zid,delivery_date,sp_code,sp_name,customer_code,customer_name,state,mobile_number
0,100000,2025-09-07,SA--000255,Sumon Miah,CUS-000368,Asif Hardware & Paint,Dhaka retail,+8801712276968
1,100000,2025-09-07,SA--000006,Muhammad Zakir Hossain,CUS-000808,Rahaman Hardware,Dhaka retail,"01819418349,01943201084"
2,100000,2025-09-07,SA--000108,Md. Anichur Rahman,CUS-001246,Al-Amin Hardware,Dhaka retail,01712271091
3,100000,2025-09-04,SA--000144,Syed Mobarok Hossain,CUS-002683,M/S Chandgaon Hardware Store,District,"01919269644,01812347618"
4,100000,2025-09-04,SA--000070,Md. Sumon Sheikh,CUS-002863,Shanda Hardware,District,+8801711888051
5,100000,2025-09-04,SA--000021,Md. Maruful Islam,CUS-003184,Riyadh Sanitary,District,01711966560
6,100000,2025-09-04,SA--000144,Syed Mobarok Hossain,CUS-004103,Giash Traders,District,01711170968
7,100000,2025-09-07,SA--000258,Md. Suruj Mia,CUS-005016,Tilottoma Electric,Dhaka retail,
8,100000,2025-09-07,SA--000092,Md. Abdullah Al Mamun Emon,CUS-007190,Jahid Enterprise,Dhaka retail,01790988886
9,100000,2025-09-04,SA--000114,Md. Pavel Mia,CUS-007414,Tarek Hardware,District,01917681570


In [23]:

file_path = "main_sheet.xlsx"

# Get file creation time
creation_time = os.path.getctime(file_path)
created_datetime = datetime.fromtimestamp(creation_time)
today_datetime = datetime.today()
cutoff_date = today_datetime - timedelta(days=60)

print("File created on:", created_datetime.strftime("%Y-%m-%d"))
print("Cutoff date (60 days ago):", cutoff_date.strftime("%Y-%m-%d"))

# ✅ Correct logic: Delete if file is OLDER than 60 days
if created_datetime < cutoff_date:
    print("🗑️ File is older than 60 days. Deleting...")
    # check if the file exist
    if os.path.exists(file_path):
        os.remove(file_path)
    else:
        print("⚠️ File does not exist.")
    print("✅ File deleted.")
else:
    print("✅ File is recent (within last 60 days). Keeping it.")

File created on: 2025-09-08
Cutoff date (60 days ago): 2025-07-10
✅ File is recent (within last 60 days). Keeping it.


In [25]:

MAIN_SHEET_PATH = "main_sheet.xlsx"
TODAY_FILE_PATH = "today_customer_order.xlsx"

# Email settings (update these with your info)


# Define expected columns
COLUMNS = [
    "zid", "delivery_date", "sp_code", "sp_name",
    "customer_code", "customer_name", "state", "mobile_number"
]

# ======================
# STEP 1: Initialize main_sheet.xlsx if not exists
# ======================
if not os.path.exists(MAIN_SHEET_PATH):
    df_empty = pd.DataFrame(columns=COLUMNS)
    df_empty.to_excel(MAIN_SHEET_PATH, index=False)
    print(f"✅ Created new main sheet: {MAIN_SHEET_PATH}")
else:
    print(f"📂 Main sheet found: {MAIN_SHEET_PATH}")

# ======================
# STEP 2: Load existing customer codes
# ======================
try:
    df_main = pd.read_excel(MAIN_SHEET_PATH, engine='openpyxl')
    if 'customer_code' in df_main.columns:
        existing_customers = df_main['customer_code'].dropna().astype(str).tolist()
        print(f"📋 Loaded {len(existing_customers)} existing customer codes.")
    else:
        existing_customers = []
        print("⚠️ No 'customer_code' column found. Assuming no existing customers.")
except Exception as e:
    print(f"❌ Error reading main sheet: {e}")
    existing_customers = []

# ======================
# STEP 3: Filter new customers from df_order_details
# ======================

# Ensure customer_code is string for safe comparison
df_order_details['customer_code'] = df_order_details['customer_code'].astype(str)

# Filter rows NOT in existing_customers → these are NEW
df_new_customers = df_order_details[~df_order_details['customer_code'].isin(existing_customers)].copy()
print(f"🆕 Filtered {len(df_new_customers)} new customer orders.")

# ======================
# STEP 4: Export today’s file & append to main sheet (Compatible with pandas 1.3.5)
# ======================
if not df_new_customers.empty:
    # Export today’s new orders
    df_new_customers.to_excel(TODAY_FILE_PATH, index=False)
    print(f"💾 Saved today's new orders to: {TODAY_FILE_PATH}")

    # Append to main sheet — compatible with pandas < 1.4.0
    try:
        # Load existing data
        df_existing = pd.read_excel(MAIN_SHEET_PATH, engine='openpyxl')
        
        # Concatenate old + new
        df_combined = pd.concat([df_existing, df_new_customers], ignore_index=True)
        
        # Write back to file (overwrites, but includes all data)
        df_combined.to_excel(MAIN_SHEET_PATH, index=False)
        
        print(f"📎 Appended {len(df_new_customers)} rows to main sheet.")
    except Exception as e:
        print(f"❌ Failed to append to main sheet: {e}")
else:
    print("📭 No new customers — skipping export and append.")

# ======================
# STEP 5: Email the file (only if new customers exist)
# ======================
if not df_new_customers.empty:
    try:
        send_mail(
            subject="HM_31: Customer Order List With Phone Number (District + Retail)",
            bodyText="Attached are the new customer orders for today.",
            attachment=[TODAY_FILE_PATH],
            recipient=["ithmbrbd@gmail.com"]
        )
        print("📧 Email sent successfully.")
    except Exception as e:
        print(f"❌ Failed to send email: {e}")
else:
    print("📭 No new customers — skipping email.")

engine.dispose()

📂 Main sheet found: main_sheet.xlsx
📋 Loaded 47 existing customer codes.
🆕 Filtered 0 new customer orders.
📭 No new customers — skipping export and append.
📭 No new customers — skipping email.
